![Banner](images/banner.png)

# cx_Oracle 8 Connection

# Architecture

Documentation reference link: [Introduction to cx_Oracle](https://cx-oracle.readthedocs.io/en/latest/user_guide/introduction.html)

![Architecture](images/architecture.png)

# Installation

Documentation reference link: [cx_Oracle 8 Installation](https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html)

**Install cx_Oracle**

Install with a command like one of the following:

```
$ python -m pip install cx_Oracle --upgrade
$ python -m pip install cx_Oracle --upgrade --user
$ python -m pip install cx_Oracle --upgrade --user --proxy=http://proxy.example.com:80
```

**Install Oracle Instant Client**

Only needed if Python is run on a computer that does **not** have Oracle Database installed.

Download and extract the Basic or Basic Light package from [oracle.com/database/technologies/instant-client.html](https://www.oracle.com/database/technologies/instant-client.html).

Make sure to download the correct architecture for your operating system.  If your Python is 32-bit, then you will need a 32-bit Instant Client.

Installation can be automated:

On Windows:
```
wget  https://download.oracle.com/otn_software/[...]/instantclient-basic-windows.x64-19.12.0.0.0dbru.zip
unzip instantclient-basic-windows.x64-19.12.0.0.0dbru.zip
```

On macOS:

```
cd $HOME/Downloads

curl -O https://download.oracle.com/otn_software/mac/instantclient/198000/instantclient-basic-macos.x64-19.8.0.0.0dbru.dmg

hdiutil mount instantclient-basic-macos.x64-19.8.0.0.0dbru.dmg

/Volumes/instantclient-basic-macos.x64-19.8.0.0.0dbru/install_ic.sh

hdiutil unmount /Volumes/instantclient-basic-macos.x64-19.8.0.0.0dbru
```

**Other Install Choices**

On Linux you can alternatively install cx_Oracle and Instant Client RPM packages from yum.oracle.com, see [yum.oracle.com/oracle-linux-python.html](https://yum.oracle.com/oracle-linux-python.html)

# Initialization

Documentation reference link: [cx_Oracle 8 Initialization](https://cx-oracle.readthedocs.io/en/latest/user_guide/initialaization.html)

When you run cx_Oracle it needs to be able to load the Oracle Client libraries.  There are several ways this can be done.

In [1]:
import cx_Oracle

import os
import sys
import platform

try:
    
    if platform.system() == "Darwin":
        cx_Oracle.init_oracle_client(lib_dir=os.environ.get("HOME")+"/instantclient_19_8")
        
    elif platform.system() == "Windows":
        cx_Oracle.init_oracle_client(lib_dir=r"C:\oracle\instantclient_19_14")
        
    # else assume system library search path includes Oracle Client libraries
    # On Linux, must use ldconfig or set LD_LIBRARY_PATH, as described in installation documentation.
    
except Exception as err:
    print("Whoops!")
    print(err);
    sys.exit(1);


# Connecting to a Database

**Connections are used for executing SQL, PL/SQL and SODA calls in an Oracle Database**

Documentation reference link: [Connecting to Oracle Database](https://cx-oracle.readthedocs.io/en/latest/user_guide/connection_handling.html)

In [2]:
# Credentials
un = "pythondemo"
pw = "welcome"

Instead of hard coding the password, you could prompt for a value, pass it as an environment variable, or use Oracle "external authentication".

### Easy Connect Syntax: "hostname/servicename"

In [3]:
cs = "localhost/orclpdb1"

connection = cx_Oracle.connect(user=un, password=pw, dsn=cs)
print(connection)

<cx_Oracle.Connection to pythondemo@localhost/orclpdb1>


Oracle Client 19c has improved [Easy Connect Plus](https://www.oracle.com/pls/topic/lookup?ctx=dblatest&id=GUID-8C85D289-6AF3-41BC-848B-BF39D32648BA) syntax:

```
cs = "tcps://my.cloud.com:1522/orclpdb1?connect_timeout=4&expire_time=10"
```

<!-- See the [technical brief](https://download.oracle.com/ocomdocs/global/Oracle-Net-19c-Easy-Connect-Plus.pdf). -->

### Oracle Network and Oracle Client Configuration Files

**Optional configuration files can be used to alter connection behaviors, such as network encryption.**

Documentation reference link: [Optional configuration files](https://cx-oracle.readthedocs.io/en/latest/user_guide/initialization.html#optional-oracle-net-configuration-files)

With the files above in `/opt/oracle/configdir`, your python application can look like:

```
# myapp.py

cx_Oracle.init_oracle_client(
    lib_dir=os.environ.get("HOME")+"/instantclient_19_3",
    config_dir="/opt/oracle/configdir"
)

connection = cx_Oracle.connect(user=un, password=pw, dsn="highperfdb")
```

## Connection Types

### Standalone Connections

Standalone connections are simple to create.

![Stand-alone Connection](images/standalone-connection.png)

In [4]:
# Stand-alone Connections

connection = cx_Oracle.connect(user=un, password=pw, dsn=cs)

print(connection)

<cx_Oracle.Connection to pythondemo@localhost/orclpdb1>


### Pooled Connections

#### Pools are highly recommended if you have:
- a lot of connections that will be used for short periods of time
- or a small number of connections that are idle for long periods of time

#### Pool advantages
- Reduced cost of setting up and tearing down connections
- Dead connection detection
- Connection- and runtime- load balancing (CLB and RLB)
- Support for Application Continuity
- Support for DRCP

![Pooled connection](images/pooled-connection.png)

In [5]:
# Pooled Connections

# Call once during application initization
pool = cx_Oracle.SessionPool(user=un, password=pw, dsn=cs, threaded=True, 
                             min=1, max=20, increment=1)

# Get a connection when needed in the application body
with pool.acquire() as connection:
    # do_something_useful(connection)
    print("Got a connection")


Got a connection


**Tip** Use a fixed size pool `min` = `max` and `increment = 0`.  See [Guideline for Preventing Connection Storms: Use Static Pools](https://www.oracle.com/pls/topic/lookup?ctx=dblatest&id=GUID-7DFBA826-7CC0-4D16-B19C-31D168069B54).

### Setting Connection "Session" State

Documentation reference link: [Session CallBacks for Setting Pooled Connection State](https://cx-oracle.readthedocs.io/en/latest/user_guide/connection_handling.html#session-callbacks-for-setting-pooled-connection-state)

Use a 'session callback' to efficiently set state such as NLS settings.

Session state is stored in each session in the pool and will be available to the next user of the session.  (Note this is different to transaction state which is rolled back when connections are released to the pool)

In [6]:

# Set some NLS state for a connection: Only invoked for new sessions
def initSession(connection, requestedTag):
    cursor = connection.cursor()
    cursor.execute("""ALTER SESSION SET NLS_DATE_FORMAT = 'YYYY-MM-DD HH24:MI' 
                                        NLS_LANGUAGE = GERMAN""")

# Create the pool with session callback defined
pool = cx_Oracle.SessionPool(user=un, password=pw, dsn=cs, 
                             sessionCallback=initSession, min=1, max=4, increment=1, threaded=True)

# Acquire a connection from the pool (will always have the new NLS setting)
with pool.acquire() as connection:
    with connection.cursor() as cursor:
        cursor.execute("""SELECT * FROM DOES_NOT_EXIST""")  # Error message is in French


DatabaseError: ORA-00942: Tabelle oder View nicht vorhanden

The callback has an optional 'tagging' capability (not shown) that allows different connections to have different state for different application requirements.

#### Callback benefit comparison

For a simple web service that is invoked 1000 times, and does 1000 queries.

![session callback comparison](images/callback-comparison.png)

### Closing Connections

Close connections when not needed.  This is important for pooled connections.

```
connection.close()
```

To avoid resource closing order issues, you may want to use `with` or let resources be closed at end of scope:

```
with pool.acquire() as connection:
    do_something(connection)
```

## Database Resident Connection Pooling

**Connection pooling on the database tier**

Documentation reference link: [Database Resident Connection Pooling (DRCP)](https://cx-oracle.readthedocs.io/en/latest/user_guide/connection_handling.html#database-resident-connection-pooling-drcp)

Dedicated server processes are the default in the database, but DRCP is an alternative when the database server is short of memory.

![DRCP architecture](images/drcp-architecture.png)

Use DRCP if and only if:
- The database computer doesn't have enough memory to keep all application connections open concurrently
- When you have thousands of users which need access to a database server session for a short period of time
- Applications mostly use same database credentials, and have identical session settings

Using DRCP in conjunction with Python Connection Pooling is recommended.

#### Memory example with 5000 application users and a DRCP pool of size 100
![DRCP memory comparison](images/drcp-comparison.png)

In Python, the connect string must request a pooled server.  For best reuse, set a connection class and use the 'SELF' purity when getting a connection from the pool.

```
pool = cx_Oracle.SessionPool(user=un, password=pw, dsn="dbhost.example.com/orcl:pooled")

connection = pool.acquire(cclass="MYCLASS", purity=cx_Oracle.ATTR_PURITY_SELF)
```

Don't forget to start the pool first!:
```
SQL> execute dbms_connection_pool.start_pool()
```

# Connecting to Autonomous Database in Oracle Cloud

If you haven't seen it, try our "Always Free" service that gives free access to Oracle DB and other cloud resources

![Banner](images/cloudhome.png)

ADB connections use "wallets" for mutual TLS to provide strong security.

Click the "DB Connection" button:

![Banner](images/adbconnection.png)

And download the wallet zip:

![Banner](images/wallet.png)

Unzip and extract the `cwallet.sso` file, and optionally the `tnsnames.ora` and `sqlnet.ora` files.

```
-rw-r--r--   1 cjones  staff   6725 15 Aug 00:12 cwallet.sso
-rw-r--r--   1 cjones  staff    134 15 Aug 10:13 sqlnet.ora
-rw-r--r--   1 cjones  staff   1801 15 Aug 00:12 tnsnames.ora
```

Keep `cwallet.sso` secure.

In [ ]:
# You still need a DB username and password.
cloud_user = "cj"
cloud_password = os.environ.get("CLOUD_PASSWORD")

# "Easy Connect" syntax can be inferred from the tnsnames.ora file entries.
# The wallet_location is the directory containing cwallet.sso.
# When using "Easy Connect", no other files from the zip are needed

# cloud_connect_string = "cjdbmelb_high"
cloud_cs = "tcps://abc.oraclecloud.com:1522/anc_cjdbmelb_high.adb.oraclecloud.com" \
           "?wallet_location=/home/cjones/CJDBMELB/"

connection = cx_Oracle.connect(user=cloud_user, password=cloud_password, dsn=cloud_cs)

with connection.cursor() as cursor:
    sql = "select user from dual"
    for r, in cursor.execute(sql):
        print("User is", r)

## STOP PRESS: Instant Client 19.14+ or 21.5+ NEWS

**You can use 1-way TLS without wallets**  See [Easy wallet-less connections to Oracle Autonomous Databases in Python](https://blogs.oracle.com/opal/post/easy-way-to-connect-python-applications-to-oracle-autonomous-databases).

**You can use Oracle IAM tokens for authentication**